カメラのflowデータはリンク数が不十分で時間帯の変動分をうまく見れないので，BLEとアンケートからq(z,x)を計算し，BLEのflowデータから推定する

そのためにflowデータを作り直す（transit probabilityではなく実数でflowデータを作ってこれをぶち込む）

0813あたりにやった操作をやり直す

In [1]:
import pandas as pd 
import numpy as np 
import os 

/Users/takahiromatsunaga/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/takahiromatsunaga/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


inputはtransit_countデータとmulti_transit_probabilityデータ

境界部分の処理に注意して実数データに変換する&ver0816とそうでないのでわける

In [122]:
basepath = '/Users/takahiromatsunaga/res2023/bledata/ble_timegroup' #/20230130_08' # /20230130_0845.csv'
day_stamp = '20230130'
time_stamp = '08'
ver_index = 'ver0815' # 'ver0815'

In [123]:
if ver_index == 'original':
    df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_node_post.csv')
    df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_link_post.csv')
    df_multiprob = pd.read_csv('/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/multi_transit_probability.csv', index_col = 0)
    dest_nodeids = [i for i in range(51, 61)]

elif ver_index == 'ver0815':
    df_node = pd.read_csv(f'/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_node_post_{ver_index}.csv')
    df_link = pd.read_csv(f'/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/ver0707/multi_link_post_{ver_index}.csv')
    df_multiprob = pd.read_csv(f'/Users/takahiromatsunaga/res2023/PPcameraTG/PP/PP_webq/q_ans/multi_transit_probability_{ver_index}.csv', index_col = 0)
    dest_nodeids = [52, 55, 56, 57, 61, 62]

nodeid_list = sorted(df_node['nodeid'].unique())

gate_nodeids = [28, 29, 35, 36, 39]
gate_nodeixs = [nodeid_list.index(i) for i in gate_nodeids]
dest_nodeixs = [nodeid_list.index(i) for i in dest_nodeids]

print(gate_nodeixs, dest_nodeixs)
# こっちにcountを紐づける
df_transit_count = pd.read_csv(f'/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/{day_stamp}_{time_stamp}/transit_counts_{day_stamp}_{time_stamp}30_20sec_off.csv', index_col=0)


[19, 20, 25, 26, 29] [30, 31, 32, 33, 34, 35]


In [124]:
# df_transit_count に39-100の間にindexを追加（0）
old_index = list(df_transit_count.index)
old_index2 = old_index[:-1]
newnodeids = old_index2 + dest_nodeids + [100]
print(newnodeids)

new_df_count = pd.DataFrame(0, index=newnodeids, columns=newnodeids)

df_transit_count.index = df_transit_count.index.astype(str)
df_transit_count.columns = df_transit_count.columns.astype(str)

new_df_count.index = new_df_count.index.astype(str)
new_df_count.columns = new_df_count.columns.astype(str)

# print(df_transit_count.index, df_transit_count.columns)
# print(new_df_count.index, new_df_count.columns)

new_df_count.update(df_transit_count)
# print(df_transit_count)
# print(new_df_count) # OK

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 52, 55, 56, 57, 61, 62, 100]


In [125]:
summ = np.sum(df_transit_count.values, axis = 0) # 各ノードへの到着数
print(summ)
print(len(summ))
nzero = len(nodeid_list) - len(summ)
extended_summ = np.concatenate([summ, np.zeros(nzero)])
print(extended_summ)

# 2. gate_nodeidsに基づいてdf_multi_probの行を選択
df_gtod_prob = df_multiprob.loc[gate_nodeids]
print('df_gtod_prob', df_gtod_prob)

extracted_summ = summ[gate_nodeixs]
print('extracted', extracted_summ)
# 3. summの各要素を対応するdf_filtered_probの行に掛け算
result_df = df_gtod_prob.multiply(extracted_summ, axis=0)

print(result_df)

result_df.index = result_df.index.astype(str)
result_df.columns = result_df.columns.astype(str)

# new_df_count.index = new_df_count.index.astype(str)
# new_df_count.columns = new_df_count.columns.astype(str)

new_df_count.update(result_df)

print(new_df_count)

if ver_index == 'original':
    new_df_count.to_csv(f'/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/{day_stamp}_{time_stamp}/joinprob_{day_stamp}_{time_stamp}30_20sec_off.csv')
elif ver_index == 'ver0815':
    new_df_count.to_csv(f'/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/{day_stamp}_{time_stamp}/joinprob_{ver_index}_{day_stamp}_{time_stamp}30_20sec_off.csv')


[   0   23  140   78    0    0    0   63  263  903 1403  366  785  617
  127    8    7  704  478  482  528  283  629  541  720  256  349  566
  649  707    0]
31
[   0.   23.  140.   78.    0.    0.    0.   63.  263.  903. 1403.  366.
  785.  617.  127.    8.    7.  704.  478.  482.  528.  283.  629.  541.
  720.  256.  349.  566.  649.  707.    0.    0.    0.    0.    0.    0.
    0.]
df_gtod_prob       1    2    3    4    5    6    7    8    9   10  ...   37   38   39  \
28  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
29  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
35  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
36  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
39  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

          52        55        56        57        61        62  100  
28  0.122642  0.084906  0.009434  0.009434  0.698113  0.075472  0.0  
2

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_7598/1461067811.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.          59.11320755
  62.55924171   0.           0.           0.           0.
 105.02564103  62.64102564   0.           0.         377.38513514
   0.           0.           0.           0.           0.
   0.           0.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_df_count.update(result_df)
/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_7598/1461067811.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0.           0.           

In [126]:
# 仕上げ


In [127]:
# # 各改札ノードから目的地ノードへの遷移確率をかけて
# sum_list = [summ[id] for id in gate_nodeixs]
# sum_list 
